# Retrieval-Augmented Generation (RAG) Project

In [1]:
pip install sentence-transformers transformers faiss-cpu --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tf-keras --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import and install libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [4]:
# Load knowledge base (FAQ)
kb = pd.read_csv("faq.csv")
print(kb)

                                             question  \
0                   What is a general health checkup?   
1         How often should I get a full body checkup?   
2                  What are the symptoms of diabetes?   
3                    How can I prevent heart disease?   
4                What is BMI and why is it important?   
5                       What vaccines do adults need?   
6                               What is telemedicine?   
7   Can I get a prescription through online consul...   
8             How can I reduce cholesterol naturally?   
9                               What is hypertension?   
10                  Are online health records secure?   
11              What is the normal blood sugar level?   
12               How much water should I drink daily?   
13  What is the difference between a virus and bac...   
14            When should I see a doctor for a fever?   
15                 What are the symptoms of COVID-19?   
16                  How can I b

In [5]:
# Embed knowledge base questions using sentence-transformers
embedder = SentenceTransformer('all-MiniLM-L6-v2')
kb_embeddings = embedder.encode(kb['question'].tolist(), convert_to_tensor=True)

C:\Users\bmoha\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [6]:
# Load generation model (T5-small fine-tuned for QA)
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
def retrieve(query, kb_embeddings, kb_df, top_k=3):
    # Embed query
    query_emb = embedder.encode(query, convert_to_tensor=True)
    # Compute cosine similarity with KB questions
    cos_scores = util.pytorch_cos_sim(query_emb, kb_embeddings)[0]
    # Get top k results
    top_results = torch.topk(cos_scores, k=top_k)
    indices = top_results.indices.cpu().numpy()
    scores = top_results.values.cpu().numpy()
    results = []
    for idx, score in zip(indices, scores):
        results.append((kb_df.iloc[idx]['question'], kb_df.iloc[idx]['answer'], score))
    return results

In [9]:
def generate_answer(query, retrieved_passages):
    # Combine retrieved answers into context
    context = " ".join([p[1] for p in retrieved_passages])
    input_text = f"question: {query} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [10]:
# Example usage:
query = "What is machine learning?"
retrieved = retrieve(query, kb_embeddings, kb, top_k=3)
print("Top retrieved documents:")
for q, a, score in retrieved:
    print(f"- Q: {q} (score: {score:.3f})")

answer = generate_answer(query, retrieved)
print("\nGenerated answer:")
print(answer)

Top retrieved documents:
- Q: What is telemedicine? (score: 0.290)
- Q: What is hypertension? (score: 0.263)
- Q: What is health insurance? (score: 0.233)

Generated answer:
to consult with doctors remotely using video, phone, or chat


In [ ]:
# Chatbot loop
print("🤖 Healthcare Assistant Chatbot (type 'exit' to stop)\n")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Bot: Thank you. Stay healthy! 👋")
        break

    retrieved = retrieve(user_input, kb_embeddings, kb)
    answer = generate_answer(user_input, retrieved)
    print("Bot:", answer)

🤖 Healthcare Assistant Chatbot (type 'exit' to stop)



You:  hypertension


Bot: high blood pressure is a chronic condition that increases risk of heart disease and stroke


You:  insurance


Bot: insurance helps cover medical expenses in exchange for a premium
